In [9]:
import os
import json
from random import shuffle
import tensorflow as tf
import tensorflow.keras as keras

In [10]:
# Load all the data from the json files
dir = r"src/dataFiveNotePatterns/"
note_patterns = []
negative_note_patterns = []

for file in os.listdir(dir):
    with open(dir + file, "r", encoding='utf-8') as f:
        data = json.load(f)
        five_notes_patterns = data["five_notes_patterns"]
        for pattern in five_notes_patterns:
            if pattern[-1] == 1:
                note_patterns.append(pattern)
            else:
                negative_note_patterns.append(pattern)





In [11]:
## make sure the two lists have the same length
if(len(note_patterns) > len(negative_note_patterns)):
    note_patterns = note_patterns[:len(negative_note_patterns)]
else:
    negative_note_patterns = negative_note_patterns[:len(note_patterns)]




In [12]:
#shuffle the data
shuffle(note_patterns)
shuffle(negative_note_patterns)

In [13]:
# Split the data into training and testing
split = int(len(note_patterns) * 0.8)
train_data = note_patterns[:split] + negative_note_patterns[:split]
test_data = note_patterns[split:] + negative_note_patterns[split:]

In [14]:
#build the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

print("Model Summary", model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Model Summary None
